In [1]:
!pip install dspy langchain_community langchain_openai dspy-ai[chromadb] jq langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dspy-ai[chromadb] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 18.4 MB/s eta 0:00:00
   ━━

In [2]:
pip install dspy-ai[chromadb] jq langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

#Set the OPENAI_API_KEY here

In [4]:
!git clone https://github.com/WAT-ai/X-Ray-Tooling.git

Cloning into 'X-Ray-Tooling'...
remote: Enumerating objects: 13435, done.
remote: Counting objects: 100% (1480/1480), done.
remote: Compressing objects: 100% (517/517), done.
remote: Total 13435 (delta 961), reused 1406 (delta 927), pack-reused 11955
Receiving objects: 100% (13435/13435), 1.14 GiB | 28.83 MiB/s, done.
Resolving deltas: 100% (5153/5153), done.
Updating files: 100% (145/145), done.


In [5]:
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

2024-07-20 15:36:19.498562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 15:36:19.498707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 15:36:19.693017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
class TreeBuilder:

    def __init__(self,
                 texts: Optional[List[str]] = None,
                 model=None,
                 embd=None,
                 random_seed: int = 224) -> None:
        self.random_seed = random_seed
        self.model = model
        self.embeddings = None
        self.texts = texts
        self.embd = embd

    def set_embedding_model(self, embd) -> None:
        self.embd = embd

    def set_language_model(self, model) -> None:
        self.model = model

    def set_texts(self, texts: List[str]) -> None:
        self.texts = texts

    def set_embeddings(self, embeddings: np.ndarray) -> None:
        self.embeddings = embeddings

    def global_cluster_embeddings(self,
                                  dim: int,
                                  n_neighbors: Optional[int] = None,
                                  metric: str = "cosine",
                                  ) -> np.ndarray:
        """
        Perform global dimensionality reduction on the embeddings using UMAP.

        Parameters:
        - dim: The target dimensionality for the reduced space.
        - n_neighbors: Optional; the number of neighbors to consider for each point.
                    If not provided, it defaults to the square root of the number of embeddings.
        - metric: The distance metric to use for UMAP.

        Returns:
        - A numpy array of the embeddings reduced to the specified dimensionality.
        """
        if self.embeddings is None:
            raise ValueError(
                "Embeddings must be set before calling this method.")

        if n_neighbors is None:
            n_neighbors = int((len(self.embeddings) - 1) ** 0.5)
        return umap.UMAP(
            n_neighbors=n_neighbors, n_components=dim, metric=metric
        ).fit_transform(self.embeddings)

    def local_cluster_embeddings(self,
                                 embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
                                 ) -> np.ndarray:
        """
        Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - dim: The target dimensionality for the reduced space.
        - num_neighbors: The number of neighbors to consider for each point.
        - metric: The distance metric to use for UMAP.

        Returns:
        - A numpy array of the embeddings reduced to the specified dimensionality.
        """
        return umap.UMAP(
            n_neighbors=num_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)

    def get_optimal_clusters(self,
                             embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 224
                             ) -> int:
        """
        Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - max_clusters: The maximum number of clusters to consider.
        - random_state: Seed for reproducibility.

        Returns:
        - An integer representing the optimal number of clusters found.
        """
        max_clusters = min(max_clusters, len(embeddings))
        n_clusters = np.arange(1, max_clusters)
        bics = []
        for n in tqdm(n_clusters, desc="Finding optimal clusters"):
            gm = GaussianMixture(n_components=n, random_state=random_state)
            gm.fit(embeddings)
            bics.append(gm.bic(embeddings))
        return n_clusters[np.argmin(bics)]

    def GMM_cluster(self, embeddings: np.ndarray, threshold: float, random_state: int = 224) -> Tuple[List[np.ndarray], int]:
        """
        Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - threshold: The probability threshold for assigning an embedding to a cluster.
        - random_state: Seed for reproducibility.

        Returns:
        - A tuple containing the cluster labels and the number of clusters determined.
        """
        n_clusters = self.get_optimal_clusters(embeddings)
        gm = GaussianMixture(n_components=n_clusters,
                             random_state=random_state)
        gm.fit(embeddings)
        probs = gm.predict_proba(embeddings)
        labels = [np.where(prob > threshold)[0] for prob in probs]
        return labels, n_clusters

    def perform_clustering(self,
                           dim: int,
                           threshold: float,
                           ) -> List[np.ndarray]:
        """
        Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
        using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

        Parameters:
        - dim: The target dimensionality for UMAP reduction.
        - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

        Returns:
        - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
        """
        if self.embeddings is None:
            raise ValueError(
                "Embeddings must be set before calling this method.")
        if len(self.embeddings) <= dim + 1:
            # Avoid clustering when there's insufficient data
            return [np.array([0]) for _ in range(len(self.embeddings))]

        # Global dimensionality reduction
        reduced_embeddings_global = self.global_cluster_embeddings(dim)
        # Global clustering
        global_clusters, n_global_clusters = self.GMM_cluster(
            reduced_embeddings_global, threshold
        )

        all_local_clusters = [np.array([])
                              for _ in range(len(self.embeddings))]
        total_clusters = 0

        # Iterate through each global cluster to perform local clustering
        for i in tqdm(range(n_global_clusters), desc="Local clustering"):
            # Extract embeddings belonging to the current global cluster
            global_cluster_embeddings_ = self.embeddings[
                np.array([i in gc for gc in global_clusters])
            ]

            if len(global_cluster_embeddings_) == 0:
                continue
            if len(global_cluster_embeddings_) <= dim + 1:
                # Handle small clusters with direct assignment
                local_clusters = [np.array([0])
                                  for _ in global_cluster_embeddings_]
                n_local_clusters = 1
            else:
                # Local dimensionality reduction and clustering
                reduced_embeddings_local = self.local_cluster_embeddings(
                    global_cluster_embeddings_, dim
                )
                local_clusters, n_local_clusters = self.GMM_cluster(
                    reduced_embeddings_local, threshold
                )

            # Assign local cluster IDs, adjusting for total clusters already processed
            for j in range(n_local_clusters):
                local_cluster_embeddings_ = global_cluster_embeddings_[
                    np.array([j in lc for lc in local_clusters])
                ]
                indices = np.where(
                    (self.embeddings ==
                     local_cluster_embeddings_[:, None]).all(-1)
                )[1]
                for idx in indices:
                    all_local_clusters[idx] = np.append(
                        all_local_clusters[idx], j + total_clusters
                    )

            total_clusters += n_local_clusters

        return all_local_clusters

    def embed(self) -> np.ndarray:
        """
        Generate embeddings for the set texts.

        This function assumes the existence of an `embd` object with a method `embed_documents`
        that takes a list of texts and returns their embeddings.

        Returns:
        - numpy.ndarray: An array of embeddings for the given text documents.
        """
        if self.texts is None:
            raise ValueError("Texts must be set before calling this method.")
        if self.embd is None:
            raise ValueError(
                "Embedding model must be set before calling this method.")
        text_embeddings = self.embd.embed_documents(self.texts)
        self.embeddings = np.array(text_embeddings)
        return self.embeddings

    def embed_cluster_texts(self) -> pd.DataFrame:
        """
        Embeds the set texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

        This function combines embedding generation and clustering into a single step.

        Returns:
        - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
        """
        if self.texts is None:
            raise ValueError("Texts must be set before calling this method.")
        self.embed()  # Generate embeddings
        cluster_labels = self.perform_clustering(
            10, 0.1
        )  # Perform clustering on the embeddings
        df = pd.DataFrame()  # Initialize a DataFrame to store the results
        df["text"] = self.texts  # Store original texts
        # Store embeddings as a list in the DataFrame
        df["embd"] = list(self.embeddings)
        df["cluster"] = cluster_labels  # Store cluster labels
        return df

    def fmt_txt(self, df: pd.DataFrame) -> str:
        """
        Formats the text documents in a DataFrame into a single string.

        Parameters:
        - df: DataFrame containing the 'text' column with text documents to format.

        Returns:
        - A single string where all text documents are joined by a specific delimiter.
        """
        unique_txt = df["text"].tolist()
        return "--- --- \n --- --- ".join(unique_txt)

    def embed_cluster_summarize_texts(self,
                                      level: int,
                                      chunk_size: int = 1500  # New parameter for chunk size
                                      ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
        clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
        the content within each cluster.

        Parameters:
        - level: An integer parameter that could define the depth or detail of processing.
        - chunk_size: The maximum length of each text chunk for summarization.

        Returns:
        - Tuple containing two DataFrames:
        1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
        2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
            and the cluster identifiers.
        """
        df_clusters = self.embed_cluster_texts()

        expanded_list = []

        for index, row in tqdm(df_clusters.iterrows(), desc="Expanding clusters", total=df_clusters.shape[0]):
            for cluster in row["cluster"]:
                expanded_list.append(
                    {"text": row["text"], "embd": row["embd"],
                        "cluster": cluster}
                )

        expanded_df = pd.DataFrame(expanded_list)
        all_clusters = expanded_df["cluster"].unique()

        print(f"--Generated {len(all_clusters)} clusters--")

        template = """Here are documents related to X-ray imaging and diagnosis.

            The information may include, but not limited to:
            - recovery time period
            - rehabilitation steps
            - restrictions during the recovery period
            - general medical advice related to x-rays
            - information about conditions and injuries diagnosable by x-rays

            Documentation:
            {context}
            """

        prompt = ChatPromptTemplate.from_template(template)

        chain = prompt | self.model | StrOutputParser()

        if self.model is None:
            raise ValueError(
                "Language model must be set before calling this method.")

        summaries = []
        for i in tqdm(all_clusters, desc="Generating summaries"):
            df_cluster = expanded_df[expanded_df["cluster"] == i]
            formatted_txt = self.fmt_txt(df_cluster)

            chunks = self.__chunk_text(formatted_txt, chunk_size)
            chunk_summaries = [chain.invoke(
                {"context": chunk}) for chunk in chunks]
            combined_summary = " ".join(chunk_summaries)
            summaries.append(combined_summary)

        df_summary = pd.DataFrame(
            {
                "summaries": summaries,
                "level": [level] * len(summaries),
                "cluster": list(all_clusters),
            }
        )

        return df_clusters, df_summary

    def __chunk_text(self, text: str, max_length: int = 1500) -> List[str]:
        """
        Chunk the given text into smaller parts each not exceeding max_length tokens.

        Parameters:
        - text: The input text to be chunked.
        - max_length: The maximum length of each chunk.

        Returns:
        - A list of text chunks.
        """
        words = text.split()
        chunks = [' '.join(words[i:i + max_length])
                  for i in range(0, len(words), max_length)]
        return chunks

    def recursive_embed_cluster_summarize(self,
                                          level: int = 1, n_levels: int = 3
                                          ) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
        """
        Recursively embeds, clusters, and summarizes texts up to a specified level or until
        the number of unique clusters becomes 1, storing the results at each level.

        Parameters:
        - level: int, current recursion level (starts at 1).
        - n_levels: int, maximum depth of recursion.

        Returns:
        - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
          levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
        """
        results = {}  # Dictionary to store results at each level

        # Perform embedding, clustering, and summarization for the current level
        df_clusters, df_summary = self.embed_cluster_summarize_texts(level)

        # Store the results of the current level
        results[level] = (df_clusters, df_summary)

        # Determine if further recursion is possible and meaningful
        unique_clusters = df_summary["cluster"].nunique()
        if level < n_levels and unique_clusters > 1:
            # Use summaries as the input texts for the next level of recursion
            new_texts = df_summary["summaries"].tolist()
            self.set_texts(new_texts)
            next_level_results = self.recursive_embed_cluster_summarize(
                level + 1, n_levels
            )

            # Merge the results from the next level into the current results dictionary
            results.update(next_level_results)

        return results

In [7]:
import os
import json
from dotenv import load_dotenv
import argparse
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import shutil
import sys
import importlib
from tqdm import tqdm
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [8]:
from abc import ABC, abstractmethod

load_dotenv(override=True)

class RaptorEmbedding():
    def __init__(
        self,
        dataset_path="X-Ray-Tooling/RAG/datasets/",
        embedding_model="text-embedding-3-small",
        llm_model="gpt-4o-mini-2024-07-18",
        level=1,
        n_levels=3
    ):

        self.__open_key = os.getenv('OPENAI_API_KEY')
        self.__embedding_open = OpenAIEmbeddings(
            openai_api_key=self.__open_key, model=embedding_model)
        self.llm = ChatOpenAI(api_key=self.__open_key,
                              model=llm_model)
        self.__persist_chroma_directory = '/kaggle/input/raptor-db/collapsed_tree_db'
        self.__processed_articles_path = dataset_path + "xray_articles_processed.json"
        self.__articles_path = dataset_path + "xray_articles.json"
        self.__tree_db = None
        if not os.path.isdir('/kaggle/input/raptor-db/collapsed_tree_db'):
            self.__process_json()
            self.__xray_articles = self.__load_xray_articles()
            self.__xray_chunked_articles = self.__chunk_documents(
                self.__xray_articles)
            self.__results = self.__build_tree(level=level, n_levels=n_levels)
            self.__collapse_tree(results=self.__results)

        self.load_tree_db()

    def __process_json(self) -> object:
        # Load the original JSON
        with open(os.path.join("/kaggle/working", self.__articles_path), "r") as file:
            data = json.load(file)

        for doc in tqdm(data, desc="Processing JSON documents"):
            doc['Authors'] = ' , '.join(doc['Authors'])
            doc['FullText'] = ' , '.join(doc['FullText'])

        with open(os.path.join("/kaggle/working", self.__processed_articles_path), "w") as file:
            json.dump(data, file, indent=4)

    def __load_xray_articles(self) -> object:
        loader = JSONLoader(
            file_path=os.path.join("/kaggle/working", self.__processed_articles_path),
            jq_schema='.[].FullText',
            text_content=True)

        return loader.load()

    def __chunk_documents(self, documents, chunk_size=1000, chunk_overlap=100) -> object:
        d_sorted = sorted(documents, key=lambda x: x.metadata["source"])
        d_reversed = list(reversed(d_sorted))
        concatenated_content = "---".join(
            [doc.page_content for doc in d_reversed]
        )
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        return text_splitter.split_text(concatenated_content)

    def __build_tree(self, level=1, n_levels=3):
        if os.path.isdir('/kaggle/input/raptor-dbcollapsed_tree_db'):
            print("Collapsed tree already exists.")
            return

        print("Building tree...")
        tree_builder = TreeBuilder(
            texts=self.__xray_chunked_articles,
            embd=self.__embedding_open,
            model=self.llm,
        )

        print("Embedding and clustering...")
        results = tree_builder.recursive_embed_cluster_summarize(
            level=level, n_levels=n_levels)

        return results

    def __collapse_tree(self, results):
        all_texts = self.__xray_chunked_articles
        for level in tqdm(sorted(results.keys()), desc="Collapsing tree"):
            summaries = results[level][1]["summaries"].tolist()
            all_texts.extend(summaries)

        vectorstore = Chroma.from_texts(
            texts=all_texts, embedding=self.__embedding_open, persist_directory=self.__persist_chroma_directory)
        self.__tree_db = vectorstore

        return

    def load_tree_db(self):
        print("Loading Collapsed Tree DB...")
        if self.__tree_db:
            print("Chroma DB already loaded.")
        else:
            vector_db = Chroma(
                persist_directory=self.__persist_chroma_directory,
                embedding_function=self.__embedding_open,
            )

            self.__tree_db = vector_db

    def get_similar_documents(self, query_text, top_n=5, search_kwargs=20, rerank=True):
        if rerank:
            print("Reranking documents...")
            return self.__rerank(query_text, top_n=top_n,
                                 search_kwargs=search_kwargs)

        return self.__tree_db.similarity_search(query_text)

    def __rerank(self, prompt, top_n=5, search_kwargs=20):
        model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
        compressor = CrossEncoderReranker(model=model, top_n=top_n)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=self.__tree_db.as_retriever(search_kwargs={"k": search_kwargs}))

        compressed_docs = compression_retriever.invoke(prompt)

        return compressed_docs

    def clear(self):
        ids_to_delete = []

        for doc in self.__chroma_db:
            if doc.metadata.get('source') == self.__xray_chunked_articles:
                ids_to_delete.append(doc.id)

        self.__tree_db.delete(ids=ids_to_delete)

    def destroy(self):
        folder_path = '/kaggle/input/raptor-db/collapsed_tree_db'
        try:
            shutil.rmtree(folder_path)
            print("Chroma DB folder deleted successfully.")
        except FileNotFoundError:
            print("The folder does not exist.")
        except Exception as e:
            print(f"An error occurred: {e}")


raptor = RaptorEmbedding(dataset_path="X-Ray-Tooling/RAG/datasets/",
                         embedding_model="text-embedding-3-small",
                         llm_model="gpt-4o-mini-2024-07-18")

print("DONE")

Loading Collapsed Tree DB...
DONE


In [9]:
import dspy
import os
import openai

In [10]:
class Passage:
    def __init__(self, text):
        self.long_text = text

In [11]:
from typing import List, Union

class raptor_retrieval_model:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
        self.embedding_model.load_tree_db()
        print("Done setup")

    def __call__(self, query: str, k:int) -> List[str]:
        docs = self.embedding_model.get_similar_documents(query_text=query, top_n=k, search_kwargs=20, rerank=False)
        return [Passage(doc) for doc in docs]

In [12]:
turbo = dspy.OpenAI(model='gpt-4o-mini-2024-07-18')
dspy.settings.configure(lm=turbo, rm=raptor_retrieval_model(raptor))

#Sanity check
retriever = dspy.Retrieve(k=6)
results = retriever("What is a fracture?").passages
#print(results)
for idx, passage in enumerate(results):
    print(f'{idx+1}]', passage.page_content, '\n')

Loading Collapsed Tree DB...
Chroma DB already loaded.
Done setup
1] , Table II compares the radiographic features of the non-fracture and fracture groups. The main differences between these groups were the clinical and radiographic features pertaining to the Mirels score (pain, lesion type, and lesion size) and the presence of a cortical breach. Patients with a fracture had significantly greater scores for all 3 of the Mirels score components (p < 0.01) as well as a greater likelihood of a cortical breach in their baseline radiographs (91.8% in the fracture group versus 22.4% in the non-fracture group, p < 0.01). , Baseline Characteristics* , Characteristic	No Fracture (N = 127)	Fracture (N = 61)	P Value†	 	Side			0.062	 	 Right	72 (56.7%)	30 (49.2%)		 	 Left	55 (43.3%)	31 (50.8%)	0.037	 	Age (yr)	69.0 ± 14.1	73.5 ± 12.9		 	Sex			0.75	 	 Female	75 (59.1%)	34 (55.7%)		 	 Male	52 (40.9%)	27 (44.3%)		 	Disease			0.022	 	 Bladder	0	1 (1.6%)		 	 Brain	2 (1.6%)	0		 	 Breast	29 (22.8%)	8 (13

In [13]:
fracture_dataset_v0 = [
    {'context': "A fracture is a medical condition where there is a partial or complete break in the continuity of a bone. Fractures occur when the physical force exerted on the bone is stronger than the bone itself. They can result from trauma, such as falls or accidents, or from conditions that weaken the bones, such as osteoporosis or certain cancers. The symptoms of a fracture typically include intense pain, swelling, bruising, and an inability to move the affected area. Fractures are classified into different types based on their characteristics, such as open (where the bone breaks through the skin), closed (where the skin remains intact), transverse, oblique, and comminuted (where the bone shatters into multiple pieces). Treatment for fractures often involves immobilizing the bone with a cast or splint, and in more severe cases, surgical intervention may be necessary to realign and stabilize the bone with metal rods, plates, or screws. Proper medical care and rehabilitation are essential to ensure the bone heals correctly and to restore full function to the injured area.", 'question': 'What are different types of fractures', 'answer': 'Open, closed, transverse, oblique, and comminuted'},
    {'context': "Leg fractures are a common type of injury that involve a break in one or more of the bones in the leg, which include the femur (thigh bone), tibia (shin bone), and fibula. These fractures can result from high-impact trauma such as car accidents, sports injuries, or falls from significant heights. Symptoms of a leg fracture often include severe pain, swelling, bruising, and an inability to bear weight on the affected leg. Depending on the severity and type of the fracture, the bone may appear deformed or protrude through the skin in cases of an open fracture. Treatment varies from immobilization with a cast or brace for minor fractures to surgical intervention involving the placement of metal rods, plates, or screws for more complex breaks. Recovery typically requires a combination of rest, physical therapy, and gradual weight-bearing exercises to restore strength and mobility. Prompt and appropriate medical care is crucial to prevent complications and ensure proper healing of the fractured leg.", 'question': 'What do gradual weight-bearing exercises do for those who suffered from fractures?', 'answer': 'They restore strength and mobility'}
]

# Make 10 beautiful examples, and then generate 50-100 more with an LLM

In [14]:
import pandas as pd
from dspy.datasets.dataset import Dataset

# Create the dataset.
class FractureDataset(Dataset):
    def __init__(self, json, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        df = pd.DataFrame(json)
        self._train = df.iloc[0:1].to_dict(orient='records')
        self._dev = df.iloc[1:].to_dict(orient='records')

dataset = FractureDataset(fracture_dataset_v0)
print(dataset.train[:2])

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

[Example({'context': 'A fracture is a medical condition where there is a partial or complete break in the continuity of a bone. Fractures occur when the physical force exerted on the bone is stronger than the bone itself. They can result from trauma, such as falls or accidents, or from conditions that weaken the bones, such as osteoporosis or certain cancers. The symptoms of a fracture typically include intense pain, swelling, bruising, and an inability to move the affected area. Fractures are classified into different types based on their characteristics, such as open (where the bone breaks through the skin), closed (where the skin remains intact), transverse, oblique, and comminuted (where the bone shatters into multiple pieces). Treatment for fractures often involves immobilizing the bone with a cast or splint, and in more severe cases, surgical intervention may be necessary to realign and stabilize the bone with metal rods, plates, or screws. Proper medical care and rehabilitation 

(1, 1)

In [15]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="Typically a vague question")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A factually correct answer")

In [22]:
class RAPTOR(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        formatted_context = [doc.page_content for doc in context]
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=formatted_context, answer=prediction.answer)

In [45]:
from dspy.teleprompt import BootstrapFewShot

# Metric could check similarity using some sort of embedding, LLM evaluation, cross-encoder

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

#Should we include context? Don't think so for now
class GoodnessJudge(dspy.Signature):
    """Judge if the answer is factually correct based on the context."""

    #context = dspy.InputField(desc="Context for the prediciton")
    question = dspy.InputField(desc="Question to be answered")
    answer = dspy.InputField(desc="Predicted answer for the question")
    ground_truth = dspy.InputField(desc="Correct answer for the question")
    how_correct = dspy.OutputField(desc="On a scale of 0 to 1, how close is the predicted answer to the correct answer? Just give the number.", prefix="Scale[0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:")

judge = dspy.ChainOfThought(GoodnessJudge)

def goodness_metric(example, pred, trace=None):
    factual = judge(
        #context=example.context,
        question=example.question, 
        ground_truth=example.answer, 
        answer=pred.answer)
    
    #Add error handling for if how_correct is now a number
    return float(factual.how_correct)

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=goodness_metric)

# Compile!
compiled_raptor = teleprompter.compile(RAPTOR(), trainset=trainset)

100%|██████████| 1/1 [00:00<00:00,  3.07it/s]

Bootstrapped 1 full traces after 1 examples in round 0.


In [46]:
# Ask any question you like to this simple RAG program.
my_question = "What are the types of fractures?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_raptor(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: What are the types of fractures?
Predicted Answer: Different types of fractures include closed, open, greenstick, comminuted, transverse, oblique, and spiral fractures.
Retrieved Contexts (truncated): [', Table II compares the radiographic features of the non-fracture and fracture groups. The main differences between these groups were the clinical and radiographic features pertaining to the Mirels sc...', 'deficit in this classification. Burst fractures take place in group A in AO classification. Fractures passing through the pedicles were added to the classification, but interpedicular separation and b...', 'number of patients, with the rates in parentheses. , This is the number of patients with another fragility fracture diagnosis code at a different anatomic location from the index fracture on a claim w...', 'to 15). , Perioperative Information , We collected demographic data and comorbidities from the patient database (Table I). Standard anteroposterior and lateral forear

In [47]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Context: Typically a vague question

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A factually correct answer

---

Context:
[1] «page_content='deficit in this classification. Burst fractures take place in group A in AO classification. Fractures passing through the pedicles were added to the classification, but interpedicular separation and bone fragments with the excess pediculolaminar junction (corner) (PLC) in the spinal canal were not included in the study of Magerl et al.. Although several classifications were proposed, Thoracolumbar Injury Classification and Severity Score (TLICS) were introduced in 2005. , This classification is based on the morphology of the injury, the status of the posterior longtidunal ligament (PLL) and neurologic examination. The energy that is generated due to axial and flexional loading in burst fractures is

In [1]:
#compiled_raptor.save("compiled_raptor.json")

NameError: name 'compiled_raptor' is not defined

In [37]:
#loaded_optimized_raptor = RAPTOR()
#loaded_optimized_raptor.load("compiled_raptor.json")

In [38]:
#my_question = "What are the types of fractures?"
#pred = loaded_optimized_raptor(my_question)

# Print the contexts and the answer.
#print(f"Question: {my_question}")
#print(f"Predicted Answer: {pred.answer}")
#print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: What are the types of fractures?
Predicted Answer: Open, closed, transverse, oblique, comminuted, and burst fractures.
Retrieved Contexts (truncated): [', Table II compares the radiographic features of the non-fracture and fracture groups. The main differences between these groups were the clinical and radiographic features pertaining to the Mirels sc...', 'deficit in this classification. Burst fractures take place in group A in AO classification. Fractures passing through the pedicles were added to the classification, but interpedicular separation and b...', 'number of patients, with the rates in parentheses. , This is the number of patients with another fragility fracture diagnosis code at a different anatomic location from the index fracture on a claim w...', 'to 15). , Perioperative Information , We collected demographic data and comorbidities from the patient database (Table I). Standard anteroposterior and lateral forearm radiographs were available for a...']


In [ ]:
###### Don't use evaluate for now

from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_devset = Evaluate(devset=devset, num_threads=1, display_progress=False, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
evaluate_on_devset(compiled_raptor, metric=goodness_metric)